In [1]:
import numpy as np
import time

from pollen_manipulation.reachy_2.api import Reachy2ManipulationAPI

from pollen_vision.camera_wrappers.depthai import SDKWrapper
from pollen_vision.camera_wrappers.depthai.utils import get_config_file_path
from pollen_vision.perception import Perception

import FramesViewer.utils as fv_utils
from reachy2_sdk import ReachySDK

import numpy as np
from pyquaternion import Quaternion
import ipywidgets as widgets
from IPython.display import display, clear_output

from scipy.spatial.transform import Rotation as R

T_world_cam = fv_utils.make_pose([0.049597, 0.009989, 0.038089], [0, 0, 0])
T_world_cam[:3, :3] = np.array([[0, 0, 1], [-1, 0, 0], [0, -1, 0]])
T_world_cam = fv_utils.rotateInSelf(T_world_cam, [-48, 0, 0])

cam = SDKWrapper(get_config_file_path("CONFIG_SR"), compute_depth=True)

K = cam.get_K()

reachy = ReachySDK(host='localhost')
manipulation_api = Reachy2ManipulationAPI(reachy, T_world_cam, K)

perception = Perception(
    camera_wrapper=cam, T_world_cam=T_world_cam, freq=10.0
)
perception.start(visualize=False)

time.sleep(1.0)
manipulation_api.turn_robot_on()
time.sleep(1.0)
manipulation_api.goto_rest_position(left=False)
manipulation_api.goto_rest_position(left=True)

/home/simsim/.virtualenvs/poc_llm_grasping/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


[06/13/24 17:21:59] WARNING  Your inference package version 0.9.13 is out of date! Please upgrade to ]8;id=177611;file:///home/simsim/.virtualenvs/poc_llm_grasping/lib/python3.10/site-packages/inference/core/__init__.py\__init__.py]8;;\:]8;id=824692;file:///home/simsim/.virtualenvs/poc_llm_grasping/lib/python3.10/site-packages/inference/core/__init__.py#35\35]8;;\
                             version 0.12.0 of inference for the latest features and bug fixes by                  
                             running `pip install --upgrade inference`.                                            

[19443010E18E782700] [9.1] [1.830] [ColorCamera(1)] [warning] Unsupported resolution set for detected camera OV9782, needs 800_P or 720_P. Defaulting to 800_P
[19443010E18E782700] [9.1] [1.830] [ColorCamera(0)] [warning] Unsupported resolution set for detected camera OV9782, needs 800_P or 720_P. Defaulting to 800_P


Cameras not initialized.


model func:  <module 'contact_graspnet_pytorch.contact_graspnet' from '/home/simsim/Pollen/ai-dev/grasping/contact_graspnet_pytorch/contact_graspnet_pytorch/contact_graspnet_pytorch/contact_graspnet.py'>
/home/simsim/.cache/huggingface/hub/models--pollen-robotics--contact_graspnet/snapshots/16f1311d6380e2e6d4394d250897585fea0258a2/checkpoints/contact_graspnet/checkpoints/model.pt
=> Loading checkpoint from local file...


In [2]:
def get_object_info():
    object_info = [info for info in perception.get_objects_infos() if info['name'] == object_name and info['detection_score']>0.7]

    if len(object_info) == 0:
        print(f'No {object_name} found')
        return []
    
    elif len(object_info) == 1:
        bottle_info = object_info[0]
        print(f'Found a {object_name}')
        return object_info[0]
    else:
        print(f'Found {len(object_info)} {object_name}')
        return []

def compute_goal_diff(fk_matrix, goal_pose):
    return np.linalg.norm(fk_matrix - selected_array)

def compute_l2_distance(fk_matrix, goal_pose):
    return np.linalg.norm(fk_matrix[:-1, 3] - goal_pose[:-1, 3])

def compute_rotation_distance(fk_matrix, goal_pose):
    q1 = Quaternion(matrix=fk_matrix[:3, :3])
    Q, R = np.linalg.qr(goal_pose[:3, :3])
    q2 = Quaternion(matrix=Q)
    return Quaternion.distance(q1, q2)

def produce_symetrical_poses(grasp_poses):
    symetrical_grasp_poses = []

    for grasp_pose in grasp_poses:
        symetrical_grasp_pose = np.eye(4)
        x, y, z = grasp_pose[:-1, 3]
        symetrical_grasp_pose[:-1, 3] = [x, -y, z]

        rotation_matrix = grasp_pose[:-1, :-1]
        roll, pitch, yaw = R.from_matrix(rotation_matrix).as_euler("xyz", degrees=True)
        symetrical_grasp_pose[:-1, :-1] = R.from_euler("xyz", [-roll, pitch, -yaw], degrees=True).as_matrix()
        symetrical_grasp_poses.append(symetrical_grasp_pose)

    return symetrical_grasp_poses

# Define the initial pose matrices for each arm used by goto_rest_position API method
right_start_pose = np.array([
    [0.0, 0.0, -1.0, 0.20],
    [0.0, 1.0, 0.0, -0.24],
    [1.0, 0.0, 0.0, -0.23],
    [0.0, 0.0, 0.0, 1.0],
])
left_start_pose = np.array([
    [0.0, 0.0, -1.0, 0.20],
    [0.0, 1.0, 0.0, 0.24],
    [1.0, 0.0, 0.0, -0.23],
    [0.0, 0.0, 0.0, 1.0],
])

In [3]:
object_name = 'mug'

perception.set_tracked_objects([object_name])

Adding tracking for object: mug


## Trigger grasping on fake robot
Takes the first reachable grasp pose in the list generated by graspnet and perform grasping.

In [5]:
object_info = get_object_info()

if object_info == []:
    print("Can't try grasping")
else:
    manipulation_api.grasp_object(object_info, left=True)
    manipulation_api.goto_rest_position(left=True, open_gripper=False)

Found a mug
Contact graspnet infer
Converting depth to point cloud(s)...
using local regions
Extracted Region Cube Size:  0.20399999618530273
Generated 100 grasps for object 1
scores: {1: array([    0.22736,     0.16856,     0.23865,     0.21731,     0.22139,     0.28431,      0.2472,      0.2314,     0.15246,     0.19024,     0.21693,     0.16738,     0.25618,     0.20256,     0.19861,     0.24572,     0.22216,     0.24216,     0.28756,     0.22147,     0.16527,     0.23074,     0.15168,     0.22004,
           0.21214,     0.25982,     0.20929,      0.2784,     0.16727,     0.21693,     0.26543,     0.16442,     0.26559,     0.16076,     0.16401,     0.15146,     0.26151,     0.25894,     0.19494,     0.15356,     0.25929,       0.234,     0.20051,     0.16296,     0.28573,     0.27184,     0.15829,     0.17951,
           0.17412,      0.2253,     0.22841,     0.20806,     0.28489,     0.16523,     0.28164,     0.19046,     0.20555,       0.279,     0.17971,      0.2438,     0.20825

In [6]:
grasp_pose_test = np.array([
    [-0.20852, 0.90069, -0.38116, 0.25596],
    [-0.97297, -0.2306, -0.012651, 0.032521],
    [-0.099292, 0.36822, 0.92442, -0.27823],
    [0, 0 , 0 , 1]
])

In [7]:
manipulation_api.reachy.l_arm.goto_from_matrix(
    grasp_pose_test,
    duration=4.0,
    with_cartesian_interpolation=True,
    interpolation_frequency=120,
)

Precision is not good enough, spamming the goal position!
l2 xyz distance to goal: 0.0001437062604641641


In [15]:
manipulation_api.goto_rest_position(left=True)

## Get all reachable grasp poses candidates

First, get the list of all the reachable grasp poses. The rest position is also inserted as first position so that it is possible to put the arm back to the rest position with the slider when viewing the grasp pose candidates.

A list with symetrical grasp poses is also generated. The symmetry of a given grasp pose for an arm is such that the other arm should end up in the same position as if it was with a mirror. Having this was useful to check the symmetry of both arms.

In [4]:
object_info = get_object_info()

if object_info == []:
    print("Can't get reachable grasp poses")
else:
    pose = object_info["pose"]
    rgb = object_info["rgb"]
    mask = object_info["mask"]
    depth = object_info["depth"]

    # Put this flag to False if you want to grasp with the right arm
    left = True

    if left:
        arm = manipulation_api.reachy.l_arm
        symetrical_arm = manipulation_api.reachy.r_arm
        start_pose = left_start_pose
        
    else:
        arm = manipulation_api.reachy.r_arm
        symetrical_arm = manipulation_api.reachy.l_arm
        start_pose = right_start_pose
    
    grasp_poses, scores, _, _ = manipulation_api.get_reachable_grasp_poses(rgb, depth, mask, left)
    grasp_poses.insert(0, start_pose)
    scores.insert(0, 1.0)
    
    symetrical_grasp_poses = produce_symetrical_poses(grasp_poses)

Found a mug
Contact graspnet infer
Converting depth to point cloud(s)...
using local regions
Extracted Region Cube Size:  0.2239999771118164
Generated 78 grasps for object 1
scores: {1: array([    0.18117,     0.15658,     0.16529,     0.17062,     0.22528,     0.15222,     0.15378,     0.20105,     0.15437,     0.18162,     0.20832,     0.16851,     0.15242,     0.18233,     0.21177,     0.17958,     0.22085,      0.1578,     0.19571,     0.18188,     0.21747,     0.17571,     0.21645,     0.15733,
            0.1667,     0.20484,     0.21929,     0.21932,     0.15143,     0.23299,     0.19457,      0.1848,     0.25274,     0.15919,     0.19077,     0.22243,     0.16211,     0.18258,     0.21369,     0.15343,     0.17309,      0.1625,     0.16657,     0.20321,     0.19596,     0.19897,      0.2043,     0.19832,
            0.2055,     0.22243,      0.1927,     0.16895,     0.17012,      0.2293,     0.15043,     0.15565,     0.15059,     0.18758,     0.21678,      0.2041,     0.21197, 

## Iterate over each reachable grasp pose candidate

Using the slider, iterate over each reachable grasp pose and send it to the fake robot to visualize what would be the movement of the arm in rviz.

Once the grasp pose reached, the l2 distance between the actual cartesian coordinates of the effector and the one from the grasp pose is computed. 

In [7]:
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(grasp_poses) - 1,
    step=1,
    description='Tableau:',
    orientation='horizontal',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='700px')  # Ajustez la largeur ici
)

# Bouton pour exécuter l'action
button = widgets.Button(description="Send goal pose")

# Zone de sortie pour afficher les résultats
output = widgets.Output()

def on_slider_change(change):
    with output:
        clear_output()
        selected_index = change['new']
        selected_array = grasp_poses[selected_index]
        print(f"Tableau {selected_index}: ")
        print(selected_array)
        print(f"Pose score: {scores[selected_index]}")

        goto_id = arm.goto_from_matrix(selected_array, with_cartesian_interpolation=True, interpolation_frequency=120, duration=2.0)

        # if goto_id != -1:
        #     while not manipulation_api.reachy.is_move_finished(goto_id):
        #         time.sleep(0.1)

        #     time.sleep(1.0)

        joints_pos = arm.get_joints_positions()
        fk = arm.forward_kinematics()
    
        l2_dist = compute_l2_distance(fk, selected_array)
        print(f'l2 dist: {l2_dist}')

        ## Uncomment the lines below to send symmetric grasp poses to the other arm

        # goto_id = symetrical_arm.goto_from_matrix(symetrical_grasp_poses[selected_index])
        # if goto_id != -1:
        #     while not manipulation_api.reachy.is_move_finished(goto_id):
        #         time.sleep(0.1)

        #     time.sleep(1.0)
        #     sym_fk = symetrical_arm.forward_kinematics()
        
        #     l2_dist = compute_l2_distance(sym_fk, symetrical_grasp_poses[selected_index])
        #     print(f'l2 dist for symetrical arm: {l2_dist}')
        #     print(symetrical_grasp_poses[selected_index])

# Associer la fonction de changement de valeur au slider
slider.observe(on_slider_change, names='value')

# Afficher les widgets
display(slider, output)

IntSlider(value=0, description='Tableau:', layout=Layout(width='700px'), max=40, style=SliderStyle(description…

Output()

In [12]:
arm.goto_from_matrix(left_start_pose, with_cartesian_interpolation=True, interpolation_frequency=120, duration=2.0)

In [10]:
manipulation_api.goto_rest_position(left=True)
manipulation_api.goto_rest_position(left=False)